<a href="https://colab.research.google.com/github/moovingsloth/video-attention-classification/blob/main/DAiSEE_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B_%EB%B6%84%EC%84%9D_%EA%B8%B0%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gcsfs tensorflow opencv-python tqdm --quiet

import gcsfs
from google.oauth2 import service_account
from google.colab import drive

drive.mount('/content/drive')

KEY_PATH = '/content/drive/MyDrive/keys/daisee-464310-abc81e54c985.json'
SCOPES = ['https://www.googleapis.com/auth/cloud-platform']

creds = service_account.Credentials.from_service_account_file(
    KEY_PATH,
    scopes=SCOPES
)
fs = gcsfs.GCSFileSystem(project='daisee-464310', token=creds)

GCS_DATASET_DIR = 'gs://colab-daisee-bucket/DAiSEE/DataSet/'
GCS_LABELS_DIR  = 'gs://colab-daisee-bucket/DAiSEE/Labels/'

print("GCSFS ready. KEY_PATH:", KEY_PATH)
print(fs.ls('colab-daisee-bucket/DAiSEE/DataSet/'))
print(fs.ls('colab-daisee-bucket/DAiSEE/Labels/'))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
GCSFS ready. KEY_PATH: /content/drive/MyDrive/keys/daisee-464310-abc81e54c985.json
['colab-daisee-bucket/DAiSEE/DataSet/', 'colab-daisee-bucket/DAiSEE/DataSet/Test', 'colab-daisee-bucket/DAiSEE/DataSet/Test.txt', 'colab-daisee-bucket/DAiSEE/DataSet/Train', 'colab-daisee-bucket/DAiSEE/DataSet/Train.txt', 'colab-daisee-bucket/DAiSEE/DataSet/Validation', 'colab-daisee-bucket/DAiSEE/DataSet/Validation.txt']
['colab-daisee-bucket/DAiSEE/Labels/AllLabels.csv', 'colab-daisee-bucket/DAiSEE/Labels/TestLabels.csv', 'colab-daisee-bucket/DAiSEE/Labels/TrainLabels.csv', 'colab-daisee-bucket/DAiSEE/Labels/ValidationLabels.csv']


In [2]:
import cv2
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE
batch_size = 32
# Haar Cascade 알고리즘(사전 학습된(Pre-trained) 얼굴 검출 모델, xml 포맷)
HAAR_PATH = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
print("Using Haarcascade:", HAAR_PATH)

Using Haarcascade: /usr/local/lib/python3.11/dist-packages/cv2/data/haarcascade_frontalface_default.xml


In [8]:
import os, random
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
import cv2
from concurrent.futures import ThreadPoolExecutor, as_completed

# 목표: 손실, 누락된 비디오 파일을 사전에 제거
# 불완전한 프레임 건너뛰기 or 제거
# 프레임 길이가 너무 짧거나 비정상인 크기 필터링
# 타임윈도잉, 슬라이딩 윈도우 (LSTM, TCN 등에 넣을 고정된 시퀀스)
# 불필요한 노이즈 제거(얼굴 검출, ROI)
# 픽셀 레벨 정규화(normalization, equalizeHist())
# Data Augmentation: 좌우뒤집기, 랜덤 크롭, 회전, 색변환 / 시간축 뒤집기
# 클래스 불균형 처리: undersampling/oversampling / class_weight 조정
# one-hot encoding, label smoothing
# TFRecord/TF.Data 파이파이프라인 구성 - 캐싱, 셔플 등등

# 시간 순서 정보를 고려해 시계열 통계 feature 추가

class DataPreprocessing:
    def __init__(self, fs, bucket: str, base_path: str = 'DAiSEE',
                 IMG_HEIGHT: int = 224, IMG_WIDTH: int = 224,
                 max_frames: int = 3, data_augmentation_flag: bool = False,
                 num_workers: int = 8):
        self.fs = fs
        self.bucket = bucket
        self.base_images = f'{bucket}/{base_path}/DataSet'
        self.base_labels = f'{bucket}/{base_path}/Labels'
        self.IMG_H, self.IMG_W = IMG_HEIGHT, IMG_WIDTH
        self.max_frames = max_frames
        self.data_augmentation_flag = data_augmentation_flag
        self.num_workers = num_workers
        # 얼굴 검출기 초기화
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        # 레이블 DataFrame 한 번만 로드
        self.label_dfs = {}
        for split in ('Train','Test','Validation'):
            path = f'{self.base_labels}/{split}Labels.csv'
            with self.fs.open(path,'rb') as f:
                self.label_dfs[split] = pd.read_csv(f)

    def get_video_paths(self, split: str) -> list:
        pattern = f'{self.base_images}/{split}/*/*/*'
        vids = [p for p in self.fs.glob(pattern) if p.lower().endswith(('.avi','.mp4'))]
        print(f"{split}: found {len(vids)} videos")
        return vids

    def extract_frames(self, video_path: str) -> list:
        # 임시 로컬 파일로 프레임 추출
        tmp = '/tmp/tmp_video.avi'
        print(f"Attempting to process video: {video_path}") # Debug print
        try:
            with self.fs.open(video_path,'rb') as src:
                content = src.read()
                if len(content) < 100 * 1024:  # ✅ 손상된 비디오 필터링
                    print(f"Warning: File too small ({len(content)} bytes): {video_path}. Skipping.")
                    return []
                with open(tmp,'wb') as dst:
                    dst.write(content)
            print(f"Downloaded {video_path} to {tmp}") # Debug print

            cap = cv2.VideoCapture(tmp)
            if not cap.isOpened():  # ✅ OpenCV가 파일을 열지 못하는 경우
                print(f"Error: Could not open video file {tmp} with OpenCV.") # Debug print
                return []

            total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            print(f"Video {tmp} has {total} frames.") # Debug print

            if total == 0:  # ✅ 프레임 수가 0인 경우
                print(f"Warning: Video {video_path} has 0 frames.") # Debug print
                return []

            indices = np.linspace(0, total - 2, self.max_frames, dtype=int)
            frames = []
            for idx in indices:
                cap.set(cv2.CAP_PROP_POS_FRAMES, int(idx))
                ret, frame = cap.read()
                if not ret:
                    print(f"Warning: Failed to read frame {idx} from {tmp}.") # Debug print
                    continue
                # BGR -> RGB 변환
                frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            cap.release()
            print(f"Successfully extracted {len(frames)} frames from {video_path}") # Debug print
        except Exception as e:
            print(f"Error processing video {video_path}: {e}")
            return [] # ✅ 예외 발생 시 안전하게 빈 리스트 반환
        finally:
            try:
                if os.path.exists(tmp):  # ✅ 임시 파일 정리
                    os.remove(tmp)
                    print(f"Cleaned up temporary file {tmp}") # Debug print
            except Exception as e:
                print(f"Warning: Failed to remove temp file {tmp}: {e}")
        return frames


    def extract_ts_features(self, frames: list) -> np.ndarray:
        """
        CNN 이미지 시퀀스를 기반으로 시계열 통계 피처 추출 (밝기, 히스토그램, 이미지 변화량 등)
        """
        if len(frames) < 2:
            return np.zeros(5, dtype=np.float32)

        # (1) 밝기 기반
        brightness_seq = []
        diff_seq = []
        for i, f in enumerate(frames):
            gray = cv2.cvtColor(f, cv2.COLOR_RGB2GRAY)
            brightness_seq.append(np.mean(gray))
            if i > 0:
                diff_seq.append(np.mean(np.abs(gray.astype(np.float32) - prev_gray.astype(np.float32))))
            prev_gray = gray

        ts = pd.Series(brightness_seq)
        diff_ts = pd.Series(diff_seq)

        return np.array([
            ts.mean(),                    # mean brightness
            ts.std(),                     # std brightness
            diff_ts.mean(),              # average frame difference
            np.max(np.gradient(ts)),     # max brightness slope
            np.std(diff_seq),            # std of frame-to-frame diff
        ], dtype=np.float32)

    def face_cropping(self, image: np.ndarray) -> np.ndarray:
        # 얼굴 검출 및 크롭, 리사이즈
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) # Face detection works better on grayscale
        faces = self.face_cascade.detectMultiScale(gray,1.3,5)
        if len(faces)>0:
            x,y,w,h = faces[0]
            # Ensure crop coordinates are within image bounds
            y = max(0, y)
            x = max(0, x)
            h = min(h, image.shape[0] - y)
            w = min(w, image.shape[1] - x)
            image = image[y:y+h, x:x+w]
        return cv2.resize(image,(self.IMG_W,self.IMG_H), interpolation=cv2.INTER_AREA)

    def augment_image(self, image: np.ndarray) -> list:
        # 간단 증강 연산 정의
        # Ensure image is in the correct format (e.g., uint8) for TensorFlow ops
        image = tf.convert_to_tensor(image, dtype=tf.float32) # Convert to float32 for augmentation ops
        ops = [
            lambda x: tf.image.flip_left_right(x),
            lambda x: tf.image.random_brightness(x,0.2),
            lambda x: tf.image.random_contrast(x,0.8,1.2)
        ]
        variants = []
        for op in ops:
            # Apply augmentation and convert back to numpy uint8
            aug = op(image)
            aug = tf.cast(aug, dtype=tf.uint8).numpy()
            variants.append(cv2.resize(aug,(self.IMG_W,self.IMG_H)))
        return variants

    def _bytes_feature(self, value: bytes) -> tf.train.Feature:
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

    def label_onehot(self, video_path: str, split: str) -> bytes:
        # ClipID 기준 레이블 조회 및 one-hot 변환
        vid = os.path.basename(video_path).rsplit('.',1)[0] + '.avi' # Ensure consistent extension
        df = self.label_dfs[split]
        row = df[df['ClipID']==vid]
        if row.empty: return None
        arr = row[['Boredom','Engagement','Confusion','Frustration ']].values[0]

        # Correctly create a one-hot encoded label
        # Find the index of the maximum value
        max_index = np.argmax(arr)
        # Create a one-hot array
        one_hot_label = np.zeros(4, dtype=np.uint8)
        one_hot_label[max_index] = 1

        return one_hot_label.tobytes()


    def _process_video_with_ts(self, video_path: str, split: str) -> list:
        """
        시계열 통계 feature를 포함한 전처리 및 TFRecord example 생성
        """
        lab = self.label_onehot(video_path, split)
        if lab is None:
            print(f"Warning: Label not found for video {video_path}. Skipping.")
            return []

        frames = self.extract_frames(video_path)
        if not frames: # Skip if no frames were extracted
            print(f"Warning: No frames extracted for video {video_path}. Skipping.")
            return []

        # 시계열 통계 feature 추출
        ts_features = self.extract_ts_features(frames).tobytes()

        examples = []
        for frame in frames:
            img = self.face_cropping(frame)
            # Ensure img is not empty after cropping (e.g., if no face was detected)
            if img is None or img.size == 0:
                print(f"Warning: Face cropping failed for a frame in video {video_path}. Skipping frame.")
                continue

            variants = self.augment_image(img) if self.data_augmentation_flag else [img]
            for v in variants:
                # Ensure augmented image is not empty
                if v is None or v.size == 0:
                     print(f"Warning: Augmentation resulted in an empty image for video {video_path}. Skipping variant.")
                     continue
                jpeg = tf.io.encode_jpeg(v).numpy()
                feat = {
                    'image': self._bytes_feature(jpeg),
                    'label': self._bytes_feature(lab),
                    'ts_feature': self._bytes_feature(ts_features),
                }
                examples.append(tf.train.Example(features=tf.train.Features(feature=feat)))
        return examples


    def _process_video_simple(self, video_path: str, split: str) -> list:
        """
        시계열 통계를 포함하지 않은 단순 전처리
        """
        lab = self.label_onehot(video_path, split)
        if lab is None:
            print(f"Warning: Label not found for video {video_path}. Skipping.")
            return []

        frames = self.extract_frames(video_path)
        if not frames: # Skip if no frames were extracted
            print(f"Warning: No frames extracted for video {video_path}. Skipping.")
            return []

        examples = []
        for frame in frames:
            img = self.face_cropping(frame)
             # Ensure img is not empty after cropping (e.g., if no face was detected)
            if img is None or img.size == 0:
                print(f"Warning: Face cropping failed for a frame in video {video_path}. Skipping frame.")
                continue
            variants = self.augment_image(img) if self.data_augmentation_flag else [img]
            for v in variants:
                 # Ensure augmented image is not empty
                if v is None or v.size == 0:
                     print(f"Warning: Augmentation resulted in an empty image for video {video_path}. Skipping variant.")
                     continue
                jpeg = tf.io.encode_jpeg(v).numpy()
                feat = {
                    'image': self._bytes_feature(jpeg),
                    'label': self._bytes_feature(lab)
                }
                examples.append(tf.train.Example(features=tf.train.Features(feature=feat)))
        return examples


    def writeTfRecord(self, output_dir: str = 'tfrecords', use_ts_feature: bool = True):
        """
        TFRecord 저장. use_ts_feature=True면 시계열 통계 포함
        """
        os.makedirs(output_dir, exist_ok=True)
        process_fn = self._process_video_with_ts if use_ts_feature else self._process_video_simple

        for split in ('Train', 'Test', 'Validation'):
            paths = self.get_video_paths(split)
            # Use a temporary file path for the TFRecord writer
            output_file = f'{output_dir}/{split.lower()}.tfrecords'
            tmp_output_file = f'{output_dir}/{split.lower()}_tmp.tfrecords'

            writer = tf.io.TFRecordWriter(tmp_output_file)
            with ThreadPoolExecutor(max_workers=self.num_workers) as exe:
                futures = {exe.submit(process_fn, vp, split): vp for vp in paths}
                for future in tqdm(as_completed(futures), total=len(paths), desc=split):
                    try:
                        for ex in future.result():
                            writer.write(ex.SerializeToString())
                    except Exception as e:
                        print(f"Error processing video {futures[future]}: {e}")
                        continue # Continue processing other videos even if one fails
            writer.close()
            # Rename the temporary file to the final output file
            os.rename(tmp_output_file, output_file)


    def decode(self, example):
        feature_spec = {
            'image': tf.io.FixedLenFeature([], tf.string),
            'label': tf.io.FixedLenFeature([], tf.string),
            'ts_feature': tf.io.FixedLenFeature([], tf.string, default_value=b'')
        }

        parsed = tf.io.parse_single_example(example, feature_spec)

        img = tf.image.decode_jpeg(parsed['image'], channels=3)
        lab = tf.io.decode_raw(parsed['label'], tf.uint8)
        lab = tf.reshape(lab, [4])

        tsf = tf.io.decode_raw(parsed['ts_feature'], tf.float32)
        tsf = tf.reshape(tsf, [-1])

        return img, tf.cast(lab, tf.float32), tsf

Successfully extracted 10 frames from colab-daisee-bucket/DAiSEE/DataSet/Train/110001/1100012031/1100012031.avi
Cleaned up temporary file /tmp/tmp_video.avi
Successfully extracted 10 frames from colab-daisee-bucket/DAiSEE/DataSet/Train/110001/1100012032/1100012032.avi


In [ ]:
# 1) TensorFlow 불러오기
import tensorflow as tf

# 2) 배치 크기 설정
batch_size = 32

# 3) DataPreprocessing 인스턴스 생성
dp = DataPreprocessing(
    fs=fs,
    bucket='colab-daisee-bucket',
    max_frames=10,  # Change max_frames to 10
    data_augmentation_flag=False,
    num_workers=8
)

# 4) TFRecord 파일 생성
print("TFRecord 생성 시작 (with ts_feature, max_frames=10)...")
dp.writeTfRecord(output_dir='tfrecords_ts_10frames', use_ts_feature=True)

# print("TFRecord 생성 시작 (without ts_feature)...")
# dp.writeTfRecord(output_dir='tfrecords_simple', use_ts_feature=False)

# 5) TFRecordDataset 로딩
print("데이터 로딩 중...")

ds_ts = tf.data.TFRecordDataset('tfrecords_ts_10frames/train.tfrecords')
ds_ts = ds_ts.map(dp.decode, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size)

# ds_simple = tf.data.TFRecordDataset('tfrecords_simple/train.tfrecords')
# ds_simple = ds_simple.map(lambda x: dp.decode(x)[:2], num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size)

Downloaded colab-daisee-bucket/DAiSEE/DataSet/Train/110001/1100012036/1100012036.avi to /tmp/tmp_video.avi
Video /tmp/tmp_video.avi has 300 frames.
TFRecord 생성 시작 (with ts_feature, max_frames=10)...
Successfully extracted 10 frames from colab-daisee-bucket/DAiSEE/DataSet/Train/110001/1100012036/1100012036.avi
Cleaned up temporary file /tmp/tmp_video.avi
Attempting to process video: colab-daisee-bucket/DAiSEE/DataSet/Train/110001/1100012037/1100012037.avi
Attempting to process video: colab-daisee-bucket/DAiSEE/DataSet/Train/110001/1100012038/1100012038.avi
Attempting to process video: colab-daisee-bucket/DAiSEE/DataSet/Train/110001/1100012041/1100012041.avi
Attempting to process video: colab-daisee-bucket/DAiSEE/DataSet/Train/110001/1100012042/1100012042.avi
Attempting to process video: colab-daisee-bucket/DAiSEE/DataSet/Train/110001/1100012045/1100012045.avi
Train: found 5482 videos
Downloaded colab-daisee-bucket/DAiSEE/DataSet/Train/110001/1100012037/1100012037.avi to /tmp/tmp_video.a

Train:   0%|          | 0/5482 [00:00<?, ?it/s]

Downloaded colab-daisee-bucket/DAiSEE/DataSet/Train/110001/1100012042/1100012042.avi to /tmp/tmp_video.avi
Video /tmp/tmp_video.avi has 300 frames.
Downloaded colab-daisee-bucket/DAiSEE/DataSet/Train/110001/1100011003/1100011003.avi to /tmp/tmp_video.avi
Video /tmp/tmp_video.avi has 300 frames.
Downloaded colab-daisee-bucket/DAiSEE/DataSet/Train/110001/1100011004/1100011004.avi to /tmp/tmp_video.avi
Video /tmp/tmp_video.avi has 300 frames.
Downloaded colab-daisee-bucket/DAiSEE/DataSet/Train/110001/1100012041/1100012041.avi to /tmp/tmp_video.avi
Video /tmp/tmp_video.avi has 300 frames.
Downloaded colab-daisee-bucket/DAiSEE/DataSet/Train/110001/1100012038/1100012038.avi to /tmp/tmp_video.avi
Error: Could not open video file /tmp/tmp_video.avi with OpenCV.
Downloaded colab-daisee-bucket/DAiSEE/DataSet/Train/110001/1100011002/1100011002.avi to /tmp/tmp_video.avi
Error: Could not open video file /tmp/tmp_video.avi with OpenCV.
Downloaded colab-daisee-bucket/DAiSEE/DataSet/Train/110001/11000

Train:   0%|          | 3/5482 [00:12<6:06:41,  4.02s/it]

Attempting to process video: colab-daisee-bucket/DAiSEE/DataSet/Train/110001/1100011012/1100011012.avi


...

Attempting to process video: colab-daisee-bucket/DAiSEE/DataSet/Train/110013/1100131012/1100131012.avi
Downloaded colab-daisee-bucket/DAiSEE/DataSet/Train/110014/1100142053/1100142053.avi to /tmp/tmp_video.avi
Video /tmp/tmp_video.avi has 300 frames.
Downloaded colab-daisee-bucket/DAiSEE/DataSet/Train/110013/1100131012/1100131012.avi to /tmp/tmp_video.avi
Video /tmp/tmp_video.avi has 300 frames.
Successfully extracted 10 frames from colab-daisee-bucket/DAiSEE/DataSet/Train/110013/1100131011/1100131011.avi
Cleaned up temporary file /tmp/tmp_video.avi


Train:  12%|█▏        | 674/5482 [21:21<2:36:28,  1.95s/it]

Attempting to process video: colab-daisee-bucket/DAiSEE/DataSet/Train/110013/1100131017/1100131017.avi
Downloaded colab-daisee-bucket/DAiSEE/DataSet/Train/110013/1100131017/1100131017.avi to /tmp/tmp_video.avi
Video /tmp/tmp_video.avi has 300 frames.
Successfully extracted 9 frames from colab-daisee-bucket/DAiSEE/DataSet/Train/110014/1100142053/1100142053.avi
Cleaned up temporary file /tmp/tmp_video.avi


Train:  12%|█▏        | 675/5482 [21:22<2:07:36,  1.59s/it]

Successfully extracted 10 frames from colab-daisee-bucket/DAiSEE/DataSet/Train/110013/1100131012/1100131012.avi
Attempting to process video: colab-daisee-bucket/DAiSEE/DataSet/Train/110013/1100131019/1100131019.avi
Attempting to process video: colab-daisee-bucket/DAiSEE/DataSet/Train/110014/1100142056/1100142056.avi


In [ ]:
print("\nWithout time-series feature:")
for imgs, labs in ds_simple.take(1): # Unpack only two values
    print("images:", imgs.shape)
    print("labels:", labs.shape)
    # print("ts_features:", tsf.shape) # This line is now commented out as tsf is not available
    # print("예시 ts_feature[0]:", tsf[0].numpy()) # This line is now commented out

In [ ]:
import matplotlib.pyplot as plt

# 샘플 배치 불러오기
for imgs, labs in ds_simple.take(1): # Unpack only two values
    sample_img = imgs[0].numpy()
    sample_label = labs[0].numpy()
    break

# 시각화
plt.figure(figsize=(4, 4))
plt.imshow(sample_img.astype('uint8'))
plt.title(f"Label (one-hot): {sample_label}")
plt.axis('off')
plt.show()

In [ ]:
import numpy as np

# 레이블 수집
all_labels = []

for imgs, labs in ds_simple: # Unpack only two values
    all_labels.extend(np.argmax(labs.numpy(), axis=1))  # one-hot → 정수 라벨로 변환

# 레이블 이름 지정 (DAiSEE 기준)
label_names = ['Boredom', 'Engagement', 'Confusion', 'Frustration']

# 분포 시각화
plt.figure(figsize=(6, 4))
plt.hist(all_labels, bins=np.arange(5) - 0.5, edgecolor='black', rwidth=0.8)
plt.xticks(range(4), label_names)
plt.title("Label Distribution in Dataset")
plt.xlabel("Label")
plt.ylabel("Count")
plt.grid(axis='y')
plt.show()

In [ ]:
# TFRecordDataset 로딩
ds_train = tf.data.TFRecordDataset('tfrecords_simple/train.tfrecords')
ds_test  = tf.data.TFRecordDataset('tfrecords_simple/test.tfrecords')
ds_val   = tf.data.TFRecordDataset('tfrecords_simple/validation.tfrecords')

ds_train = ds_train.map(dp.decode).batch(batch_size)
ds_test  = ds_test.map(dp.decode).batch(batch_size)
ds_val   = ds_val.map(dp.decode).batch(batch_size)

def get_label_distribution(dataset):
    all_labels = []
    for _, labs, _ in dataset:
        all_labels.extend(np.argmax(labs.numpy(), axis=1))
    return np.array(all_labels)


# 레이블 이름
label_names = ['Boredom', 'Engagement', 'Confusion', 'Frustration']

# 분포 수집
train_labels = get_label_distribution(ds_train)
test_labels  = get_label_distribution(ds_test)
val_labels   = get_label_distribution(ds_val)

# 시각화
fig, axes = plt.subplots(1, 3, figsize=(18, 4))
splits = ['Train', 'Test', 'Validation']
datasets = [train_labels, test_labels, val_labels]

for i, (labels, split) in enumerate(zip(datasets, splits)):
    axes[i].hist(labels, bins=np.arange(5) - 0.5, edgecolor='black', rwidth=0.8)
    axes[i].set_xticks(range(4))
    axes[i].set_xticklabels(label_names)
    axes[i].set_title(f"{split} Set Label Distribution")
    axes[i].set_xlabel("Label")
    axes[i].set_ylabel("Count")
    axes[i].grid(axis='y')

plt.tight_layout()
plt.show()

In [ ]:
from collections import Counter

for name, labels in zip(['Train', 'Test', 'Validation'], [train_labels, test_labels, val_labels]):
    print(f"\n{name} Set Label Counts:")
    for i, count in sorted(Counter(labels).items()):
        print(f"  {label_names[i]}: {count}")

In [ ]:
from tensorflow.keras import layers, models

def build_model():
    model = models.Sequential([
        layers.Input(shape=(224, 224, 3)),
        layers.Rescaling(1./255),
        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(4, activation='softmax')
    ])
    return model

model = build_model()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
from tensorflow.keras import layers, models
import tensorflow as tf

def build_functional_model(input_shape, batch_size):
    # Define the input layer with a specified batch size
    inputs = tf.keras.Input(shape=input_shape, batch_size=batch_size)

    # Add Rescaling
    x = layers.Rescaling(1./255)(inputs)

    # Add Convolutional and MaxPooling layers
    x = layers.Conv2D(32, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(2, 2)(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(2, 2)(x)

    # Explicit Reshape before the dense layers
    # Calculate the flattened size: 54 * 54 * 64
    flattened_size = 54 * 54 * 64
    x = layers.Reshape((flattened_size,))(x) # Reshape to (batch_size, flattened_size)

    # Add Dense layers with Dropout
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(4, activation='softmax')(x)

    # Create the model
    model = models.Model(inputs=inputs, outputs=outputs)

    return model

# Define input shape and batch size
input_shape = (224, 224, 3)
batch_size = 32

# Build and compile the model
model = build_functional_model(input_shape, batch_size)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
for img, lab in ds_simple.take(1):
    print("image shape:", img.shape)   # (32, 224, 224, 3)
    print("label shape:", lab.shape)   # ✅ 반드시 (32, 4) 여야 함
    # print("tsf shape:", tsf.shape)     # (32, 0) 또는 제외 가능

In [ ]:
ds_val = tf.data.TFRecordDataset('tfrecords_simple/validation.tfrecords')
ds_val = ds_val.map(lambda x: dp.decode(x)[:2], num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size)

In [ ]:
for img, label in ds_simple.take(1):
    print("image shape:", img.shape)

In [ ]:
history = model.fit(ds_simple,
                    validation_data=ds_val,
                    epochs=5)

In [ ]:
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title("Model Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title("Model Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, InputLayer, Rescaling

# Build a partial model up to the last MaxPooling2D layer
partial_model = Sequential([
    InputLayer(input_shape=(224, 224, 3)),
    Rescaling(1./255),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2)
])

# Print the output shape of the partial model
print("Output shape of the last MaxPooling2D layer:", partial_model.output_shape)